# Key-Value Stores (Redis)


# Introduction:

### Redis:
   <a href="https://redis.io/">Redis</a> Redis is an open source (BSD licensed), in-memory data structure store, used as a database, cache and message broker. It supports data structures such as strings, hashes, lists, sets, sorted sets with range queries, bitmaps, hyperloglogs, geospatial indexes with radius queries and streams.<br/>
   
<img src="https://www.zend.com/sites/zend/files/image/2019-09/logo-redis.jpg" width ="250" >


#### <a href='https://redislabs.com/redis-enterprise/data-structures/'>Redis Data Structures</a>
* Redis is not a plain key-value store, it is actually a data structures server, supporting different kinds of values.
* An introduction to Redis data types and abstractions https://redis.io/topics/data-types-intro
* Redis keys are always strings.


<img src='https://redislabs.com/wp-content/uploads/2020/06/key-value-data-stores-2-v2-920x612.png' width='500' >

### How To Query Redis!

- Commands for each data type for common access patterns, with bulk operations, and partial transaction support.

### PreLab

#### 1. Install Redis on Windows
- Redis is a cross-platform DB, We can install it on Linux, or Windows, ..etc.
- There are two ways to install Redis under Windows
    - Download the latest Redis .msi file from https://github.com/MSOpenTech/redis/r... and install it. 
    
    - You can choose either from these sources
        - https://github.com/microsoftarchive/redis/releases or
        - https://github.com/rgl/redis/downloads

- Personally I prepared the first option
- Download Redis-x64-2.8.2104.zip
- Extract the zip to the prepared directory
- Run redis-server.exe
- Run redis-cli.exe
- For more info follow this setup-video tutorial (https://www.youtube.com/watch?v=188Fy-oCw4w)


#### Linux and Debian 

- Even quicker and dirtier instructions for Debian-based Linux distributions are as follows:
    - download Redis from http://redis.io/download 
    - extract, run make && sudo make install
    - Then run sudo python -m easy_install redis hiredis (hiredis is an optional performance-improving C library).

#### 2. Install the Python Package ("<a href='https://pypi.org/project/redis/'>redis</a>") to connecto to Redis 
- use th command ```pip install redis``` in your command line.


#### (more) Accessing Redis from Command Line:
- Add the Redis installation "/home" and "/bin" directories to the environment variables.
- start Redis server in one command window(CMD, poweshell, ..etc)using the command ```redis-server```.
- In another command window, start your Redis Client using the command ```redis-cli```
- Now you have the Redis Client Shell connected to the default <b>db0</b> DB. 

In [58]:
! pip install redis

In [59]:
import redis
from pprint import pprint
import pandas as pd
from time import sleep

import warnings
warnings.filterwarnings('ignore')

##### Get a client connection to redis server, using the url and the port, and Db

In [60]:
r = redis.Redis(host='localhost', port=6379, db=0)

## Task 0: First Steps in Redis

### Demo the string keys

In [61]:
r.set('language','Python')

True

##### Key expiration (e.g Think about Sessions management )


- By default, keys are retained, but we can make our keys (data) vanish after a specified time.
- This can be set while creating the key, or for already existing keys.

In [62]:
## use expire(key,time in_secs), after this time key will vanish
print(r.expire('language', 6))
#ttl(key) time_to_live, checks remaining time to live! 
print(r.ttl('language'))
sleep(3)
print(r.ttl('language'))

True
6
3


#### Check if the key already expired!
- Use exists(key) function

In [63]:
r.set('books','roman')
key_status = 'Alive' if r.exists('books') else 'Not alive'
print(key_status)

Alive


#### Setting multiple String Keys, values 

In [64]:
r.mset({"Croatia": "Zagreb", "Bahamas": "Nassau"})

True

#### Get the value of the key 'Croatia'

In [65]:
r.get("Croatia")
r.mget(["Croatia", "Bahamas"])

[b'Zagreb', b'Nassau']

#### Set String as JSON value 

In [66]:
r.set('myJsonData' , '{"name": "Ragab", "age": 40}')

True

#### Get the previous JSON value

In [67]:
r.get('myJsonData')

b'{"name": "Ragab", "age": 40}'

#### Rename the key 'myjsonData' into 'myJsonInfo'

In [68]:
r.rename('myJsonData', 'myJsonInfo')

True

#### Delete 'myJsonInfo' key-value pair.

In [69]:
r.delete('myJsonInfo')

1

#### Check if it's deleted already !

In [70]:
# we try to get the value of that key!
print(r.get('myJsonInfo'))

#or we can check if it's not existing any more !
print(r.exists ('myJsonInfo'))

None
0


### Demo The Lists

- Think of Lists as ordered sequence of strings like java ArrayList, javasrcript array, or python n lists.
- We can use lists to implement stacks and queues.
    - If you need a **Queue**, just use **RPUSH** and **LPOP**.
    - If you need a **Stack**, just use **RPUSH** and **RPOP**.
- Lists can **accept duplicates**.
- A single Resid List can hold over **4B** entries!!


#### Create a List of customers, and add elements to it!

In [71]:
r.lpush('customers','Ragab')

8

This will return "1" meaning the list now contains only one element

In [72]:
r.lpush('customers','Riccardo')

9

In [73]:
r.lpush('customers','Riccardo')

10

#### Get current memebrs of the list
- Hint: use <code>lrange</code> function

In [74]:
r.lrange('customers', 0, 1)

[b'Riccardo', b'Riccardo']

* We can clearly notice that **LPUSH** function/command adds elements to the left of the list.
* and that Lists also **accept duplicate** items.

#### Adding element to the Right of the customers list
- Add customer 'Kim' to the right of the list
- Use <code>rpush</code> function

In [75]:
r.rpush('customers', 'Kim')

11

In [76]:
#check the list elements again
r.lrange('customers',0,-1)

[b'Riccardo',
 b'Riccardo',
 b'Ragab',
 b'Riccardo',
 b'Jan',
 b'Ragab',
 b'Riccardo',
 b'Riccardo',
 b'Ragab',
 b'Jan',
 b'Kim']

#### Insert 'Jan' between Riccardo and Ragab

In [77]:
r.linsert('customers','BEFORE','Ragab','Jan')

12

In [78]:
r.lrange('customers',0,-1)

[b'Riccardo',
 b'Riccardo',
 b'Jan',
 b'Ragab',
 b'Riccardo',
 b'Jan',
 b'Ragab',
 b'Riccardo',
 b'Riccardo',
 b'Ragab',
 b'Jan',
 b'Kim']

#### Get only the first element 

In [79]:
r.lrange('customers', 0, 0)[0]
r.lindex('customers', 0)

b'Riccardo'

#### Get only the first 3 elements  

In [80]:
r.lrange('customers',0,2)

[b'Riccardo', b'Riccardo', b'Jan']

#### Get the Length of the List
- Use <code>llen</code> function

In [81]:
r.llen('customers')

12

#### Delete the first element on the left

In [82]:
r.lpop('customers')

b'Riccardo'

In [83]:
#check the list elements again
r.lrange('customers',0,-1)

[b'Riccardo',
 b'Jan',
 b'Ragab',
 b'Riccardo',
 b'Jan',
 b'Ragab',
 b'Riccardo',
 b'Riccardo',
 b'Ragab',
 b'Jan',
 b'Kim']

#### Delete the first element on the right

In [84]:
r.rpop('customers')

b'Kim'

In [85]:
#check the list elements again
r.lrange('customers',0,-1)

[b'Riccardo',
 b'Jan',
 b'Ragab',
 b'Riccardo',
 b'Jan',
 b'Ragab',
 b'Riccardo',
 b'Riccardo',
 b'Ragab',
 b'Jan']

#### Notes on the performance of Lists:
- **LPOP**, **LPUSH**, and **LLEN** commands are all **O(n)** cosnstant time operations. 
    - Their performance is independent of the lenght of the list.
- **LRANGE** is **O(s+n)**, such that **s** is the distance of the start offest from the head, and **n** is the number of the elements in the specified range.
    - Thus, we need to be careful with LRANGE especially with extra long lists, or when we retrieve thousands or more elements!!
<img src='ListsPerformance.JPG' width= '200'>

### Demo The Sets

- **Unordered** collection of strings.
- Contains **no duplicates**.
    - This makes Sets supernatual option for **de-dupication** applications.
- Questions that we can Answer using Sets:
    - **Did I see this IP address in the last hour?**
    - **Is this user online?**
    - **Has this URL been balcklisted?**
- All of these questions can be answered in **O(1)** time.
- Sets support standard operations:
    - **Intersection** 
    - **Difference**
    - **Union**

#### Create a Set of online players with the key "players:online" ["Riccardo", and "Ragab"]

In [86]:
r.sadd('players:online',"Riccardo","Ragab")

1

#### Try to another Online-player "Ragab" in the Set
- Write down what did you noticed?!

In [87]:
r.sadd('players:online',"Sang Nguyen")

0

#### Check if the Set of Online Players contain the player "Riccardo"

In [88]:
r.sismember('players:online',"Riccardo")

1

#### Check if the Set of Online Players contain the player "Fabiano"

In [89]:
r.sismember('players:online',"Fabiano")

0

#### Create Another Set with the key-name ("Friends") that has ["Riccardo", "Fabiano", "Hassan"]

In [90]:
r.sadd('friends', 'Riccardo', 'Fabiano', 'Hassan')

1

#### Get the two lists memebers  

In [91]:
print ("FirstSet:" ,r.smembers('players:online'))
print ("SecondSet:",r.smembers('friends'))

FirstSet: {b'Riccardo', b'Ragab', b'Sang Nguyen'}
SecondSet: {b'Riccardo', b'Fabiano', b'Hassan'}


#### Get the intersection of these two sets 

In [92]:
#Intersction
print(r.sinter('players:online','friends'))

{b'Riccardo'}


#### Get the Union of these two sets 

In [93]:
print(r.sunion('players:online', 'friends'))

{b'Riccardo', b'Sang Nguyen', b'Hassan', b'Ragab', b'Fabiano'}


#### Get the Length of the two Sets 

In [94]:
#Length
print(r.scard('friends'))
print(r.scard('players:online'))

3
3


#### Move "Fabiano" to the Online Players Set

In [95]:
r.smove('friends', 'players','Fabiano')

True

#### Get the Length of the two Sets After this move!

In [98]:
print(r.scard('friends'))
print(r.scard('players'))

2
1


#### Remove "Ragab" from the players Set, and show the palyers Set after this removal

In [40]:
r.srem('players:online',"Ragab")
r.smembers('players:online')

{b'Riccardo', b'Sang Nguyen'}

### Demo The SORTED SETS
- REDIS sorted sets are **ordered** collections of unique members.
- These memebrs are ordered according to their **asociated score**.
- Whenever you add to the sorted set, you are specifying a **memeber** and a **score**.
- Sorted Sets keep every thing sorted from the begininng.
- Sorted sets are good choice for:
    - **priority queues**
    - **Low-latency leaderboards**
    - **Secondary indexing**
- Questions that we can Answer using Sorted Sets (e.g, in an online-game ):
    - **Who are the top 10 players?
    - **what is the rank of a specific Player?
    - **what is the current score of the player?

#### Let's Create our Leaderboard
- In the scenario of online game, each player will have a score of '**experience**' achieving some tasks/goals,..etc.

#### Initially, Let's give a score of 0 experience to all of our players:
- We have three Players ("Ragab", "Fabianno", and "Riccardo")

In [41]:
r.zadd('players:exp',{'Ragab':0})
r.zadd('players:exp',{'Riccardo':0})
r.zadd('players:exp',{'Fabiano':0})

1

In [102]:
r.zrange('players:exp',0,2)

[b'Ragab', b'Riccardo', b'Fabiano']

#### Increment the experiernce score of our players
Let's pretend that our players have copleted some missions and they got these experience points 40,60,80 for "Ragab", "Riccardo", and "Fabiano" respectively

In [42]:
print(r.zincrby('players:exp',40,'Ragab'))
print(r.zincrby('players:exp',60,'Riccardo'))
print(r.zincrby('players:exp',80,'Fabiano'))

40.0
60.0
80.0


#### Let's Punish one of the players penalizing him with 5 points of experience

In [103]:
print(r.zincrby('players:exp',-5,'Ragab'))

35.0


#### Get the Top 3 players in our game

In [104]:
r.zrevrange('players:exp',0,2)

[b'Fabiano', b'Riccardo', b'Ragab']

#### GET the Top 3 players in our game showing their scores

In [111]:
top3 = r.zrevrange('players:exp', 0, 2, withscores=True)
for rank, (players, score) in enumerate(top3, start=1):
    print(f'{rank}. {players.decode()} - {score:.1f} XP')

1. Fabiano - 80.0 XP
2. Riccardo - 60.0 XP
3. Ragab - 35.0 XP


#### Get the Ranak of the players "Ragab", and "Fabiano"
- Look at the difference between **Zrank**, and **zrevrank**

In [46]:
print(r.zrevrank('players:exp', 'Ragab'))
print(r.zrevrank('players:exp', 'Fabiano'))

2
0


#### Get the score of the player "Riccardo"

In [112]:
print (r.zscore('players:exp', 'Riccardo'))

60.0


### Demo the HASHES 
- Hashes are one of the most useful Redis data structures.
- Hashes are collections of field-value pairs.

#### Let's Create a Hash of Players in an online game

- Each player has the following fields:
    - NAME
    - RACE
    - LEVEL
    - HEALTH
    - GOLD

#### Let's Create our first player ('player:101')

In [48]:
r.hset('player:101','name','Cyclops')
r.hset('player:101','race','Elf')
r.hset('player:101','level',4)
r.hset('player:101','health',20)
r.hset('player:101','gold',500)

1

#### Adding another player (player:102)
- We can use <code>HMSET</code> value pairs to the Hash.

In [113]:
player2 = {"name":"Wolverine", 
 "race":"Elf", 
 "level":6, 
 "health":200, 
 "gold":4000}
#We use HMSET for ading multi-field value pairs to the Hash
r.hmset('player:102', player2)

True

#### Get the information of the player Hashes ('player:101', 'player:102' )

In [50]:
pprint(r.hgetall('player:101'))
print("\n")
pprint(r.hgetall('player:102'))

{b'gold': b'500',
 b'health': b'20',
 b'level': b'4',
 b'name': b'Cyclops',
 b'race': b'Elf'}


{b'gold': b'4000',
 b'health': b'200',
 b'level': b'6',
 b'name': b'Wolverine',
 b'race': b'Elf'}


#### Get the **name** of the second player
- Use <code>hget(Hash_key,field)</code>

In [114]:
r.hget('player:102', 'gold')

b'4000'

#### Get the name, level and the race of the second player
- Use <code>hmget(Hash_key,field1,field2,..)</code>

In [116]:
r.hmget('player:102','health', 'level', 'name')

[b'200', b'6', b'Wolverine']

#### updating the Hash with adding a new field
- For player ('player:101'), add the **status** as '**Killed**' 

In [117]:
r.hset('player:101', 'status', 'Killed')

1

#### Check if added field 'status' to the first player ('player:101')

In [118]:
pprint(r.hgetall('player:101'))

{b'gold': b'500',
 b'health': b'20',
 b'level': b'4',
 b'name': b'Cyclops',
 b'race': b'Elf',
 b'status': b'Killed'}


#### updating the Hash with deleting the  'status' field for 'player:101'

In [119]:
r.hdel('player:101','status')

1

#### Check if the  field 'status' is deleted from the first player ('player:101')

In [120]:
pprint(r.hgetall('player:101'))

{b'gold': b'500',
 b'health': b'20',
 b'level': b'4',
 b'name': b'Cyclops',
 b'race': b'Elf'}


#### In such games, players recieve gold points, after completing objectives, or defeating enemies.
- Let's add some gold points to player:102 "Wolverine", Increase him by 25 points. 

In [57]:
print("Gold Before: ")
pprint(r.hget('player:102', 'gold'))

r.hincrby('player:102', 'gold', 25)

print("\nAfter: ")
pprint(r.hget('player:102', 'gold'))

Gold Before: 
b'4000'

After: 
b'4025'


**Notes on incrementing & decrementing Hash values:** 
- HINCRBY still operates on a hash value that is a string, but it tries to interpret the string as a **base-10 64-bit signed integer** to execute the operation.

- This applies to other commands related to incrementing and decrementing other data structures, namely **INCR**, **INCRBY**, **INCRBYFLOAT**, **ZINCRBY**, and **HINCRBYFLOAT**.

- You’ll get an error if the string at the value can’t be represented as an integer.


#### Notes on the performance of Redis Hashes:
- HGET, HSET, HINCRBY, HDEL are O(1) constant time opeations regardless of the size of the Hash.
- Whreas, HGETALL is O(n),with n being the number of fields in the Hash.
    - In Big Hashes of Thousands of fields, it's usually more effiecient to specify the fields you want, rather than retreiving all of the fields.
    
<img src='HashPerformance.JPG' width= '200'>